## Import Libraries

In [1]:
import pandas as pd
import networkx as nx
import math
import os

## Create Graph (DiGraph=Directed Graph)

In [2]:
G=nx.DiGraph()

## Define the Node Structure

In [3]:
class node:
    
    def __init__(self,identifier,parent_identifier,category,name,G):
        assert type(identifier)==str
        assert type(parent_identifier)==str
        assert type(category)==str
        assert type(name)==str
        
        self.name=name.strip()
        self.category=category
        self.identifier=parent_identifier+":"+identifier
        self.parent=parent_identifier
        self.children=[]
        self.attributes={}#Attributes: description, comments_list, grade, primary_lds
        
    def add_attributes(self,attributes):
        assert type(attributes)==dict
        self.attributes=attributes
            
    def insert_into_graph(self,G):
        G.add_node(self.identifier,link=self)
        
    def add_edge(self,child_identifier,category,G):
        assert type(child_identifier)==str
        assert type(category)==str
        self.children.append(child_identifier)
        G.add_edge(self.identifier,child_identifier,category=category)
        
    def parse_graphML(self):
#         string="\t\t<node id=\"%s\" labels=\"Concept\">\n"%(self.identifier)
        string="\t\t<node id=\"%s\">\n"%(self.identifier)
        string+="\t\t\t<data key=\"labels\">Concept</data>\n"
        string+="\t\t\t<data key=\"uid\">%s</data>\n"%(self.identifier)
        string+="\t\t\t<data key=\"name\">%s</data>\n"%(self.name)
        string+="\t\t\t<data key=\"objectType\">%s</data>\n"%(self.category)
        string+="\t\t\t<data key=\"parentIdentifier\">%s</data>\n"%(self.parent)
        string+="\t\t\t<data key=\"childrenIdentifiers\">%s</data>\n"%(",".join(self.children))
        for i in self.attributes.keys():
            temp=[]
            if(type(self.attributes[i])==list):
                temp=",".join(self.attributes[i])
            else:
                if(type(temp)==float and math.isnan(temp)):
                    continue
                temp=self.attributes[i]
            string+="\t\t\t<data key=\"%s\">%s</data>\n"%(i,temp)
        string+="\t\t</node>\n"
        return string

## Defining a Dictionary to store the mapping between Name and Code since the entities are referred to by their name rather than code in the funtoot documents

In [4]:
obj_dict={}

## Intialise a Level in the Graph (I tend to think of it as more of a web stretching radially outwards allowing for hierarchy and within level connections)

In [5]:
def initialise_level(identifier,category,name,parent_name,attributes,edge_category,mapping,G):
    assert type(identifier)==list
    assert type(category)==str
    assert type(name)==list
    assert type(parent_name)==list or type(parent_name)==str
    assert (type(attributes)==list and type(attributes[0])==dict) or attributes==None
    assert type(edge_category)==str
    assert type(mapping)==dict
    
    parent_identifier=[]
    if(type(parent_name)==str):
        l=min(len(identifier),len(name))
        for i in range(0,l):
            parent_identifier.append(mapping[parent_name.strip()])
    else:
        l=min(len(identifier),len(name),len(parent_name))
        for i in range(0,l):
            parent_identifier.append(mapping[parent_name[i].strip()])
    for i in range(0,l):
        parent_obj=G.node[parent_identifier[i]]["link"]
        obj=node(identifier[i],parent_identifier[i],category,name[i],G)
        if(attributes!=None and len(attributes[i].keys())>0):#If no attributes are to be defined
            obj.add_attributes(attributes[i])
        obj.insert_into_graph(G)
        mapping[obj.name]=obj.identifier
        parent_obj.add_edge(obj.identifier,edge_category,G)

## Make Connections in a Level to define the notion of Pre Requisites

In [6]:
def add_level_connections(name,prereq_name,mapping,G):
    assert type(name)==list
    assert type(prereq_name)==list
    assert type(mapping)==dict
    
    for i,j in zip(name,prereq_name):
        if(type(j)==float and math.isnan(j)):#Handle NaN values: The second expression is not evaluted until the first
            #Thus math.isnan(j) is only evaluated for float objects and not string objects(which would cause an exception)
            continue
        G.add_edge(mapping[j.strip()],mapping[i.strip()],category="Pre-Requisite")

## Defining the centre of the web

In [7]:
numeracy=node("Num","Dom","Domain","Numeracy",G)
numeracy.insert_into_graph(G)
obj_dict["Numeracy"]="Dom:Num"

## Process KG_Math

In [8]:
# The next level is that of the Areas (named Groups) defined in KG_Maths
table=pd.read_csv("Funtoot/KG_Maths/ConceptList-Table 1.csv")
area_names=list(table["Group"].drop_duplicates().dropna().get_values())
initialise_level(["AR","GE","CC","ST"],"areas",area_names,"Numeracy",None,"Child",obj_dict,G)

# Then come the Dimensions also defined in the same file
dimension_identifiers=list(table["Concept Code"].dropna().get_values())
dimension_names=list(table["Concepts in Mathematics"].dropna().get_values())
dimension_parents=list(table["Group"].dropna().get_values())
initialise_level(dimension_identifiers,"dimension",dimension_names,dimension_parents,None,"Child",obj_dict,G)

# Now let us define the Pre-Requisites between Dimensions
table=pd.read_csv("Funtoot/KG_Maths/ConceptDependency-Table 1.csv",header=1)
dimension=list(table["Concept"])
prereq_dimension=list(table["Dependency"])
add_level_connections(dimension,prereq_dimension,obj_dict,G)

#Then let us add the Misconceptions Categories associated with Numeracy
table=pd.read_csv("Funtoot/KG_Maths/LDs-Table 1.csv",header=1)
category_names=list(table["Class Category"].drop_duplicates().dropna().get_values())
category_ids=["MCN"+str(i) for i in range(0,len(category_names))]
initialise_level(category_ids,"Misconception Class",category_names,"Numeracy",None,"Induces",obj_dict,G)

#Finally let us add the Miconceptions in a given category
misconception_names=list(table["Error Category"].dropna())
misconception_parents=list(table["Class Category"].dropna())
misconception_identifiers=list(table["LD Code"].dropna())
misconception_attributes=[{"Description":i} for i in list(table["Teacher / Parent Description"].dropna())]
initialise_level(misconception_identifiers,"Misconceptions",misconception_names,misconception_parents,
                 misconception_attributes,"Child",obj_dict,G)

# Process KG_(Rest)

In [9]:
#Get main domains
dir_list=[]
for root, dirs, files in os.walk("Funtoot", topdown=False):
    for name in dirs:
        if(name.find("KG")==0 and name.find("_Maths")==-1):
            dir_list.append(name[3:])

In [10]:
for dirname in dir_list:

    #In a given dimension we have several concepts (or in Funtoot parlance known as Subconcepts)
    table=pd.read_csv("Funtoot/KG_%s/Scope-Table 1.csv"%(dirname),header=1)
    concept_names=list(table["Subconcepts"].drop_duplicates().dropna().get_values())
    concept_identifiers=["C"+str(i) for i in range(0,len(concept_names))]
    initialise_level(concept_identifiers,"Concepts",concept_names,dirname,None,"Child",obj_dict,G)  

    #For each concept we have several microconcepts
    microconcept_names=list(table["Sub Sub Concept"].dropna().get_values())
    microconcept_identifiers=["MCT"+str(i) for i in range(0,len(microconcept_names))]
    concept_names=list(table["Subconcepts"].dropna().get_values())
    l=len(microconcept_names)
    #Attributes of a given microconcept
    microconcept_attributes=[]
    #Description
    for i in range(0,l):
        microconcept_attributes.append({"description":table["Scope Items"][i]})
    comment_flag=False#Explained later
    for key in table.keys():
        #Grade
        if(key.lower().find("grade")==0):
            for i in range(0,l):
                microconcept_attributes[i]["grade"]=table[key][i]
        #Primary LDs
        elif(key.lower().find("ld")==0):
            for i in range(0,l):
                microconcept_attributes[i]["ld"]=table[key][i]
        #If the column is completely filled with NaN values then that column's dtype is not object
        #Comments follow after a series of NaN valued columns and thus the way to detect them is to
        #Detect these NaN valued columns and keep on the alert for any object dtype columns thereafter
        #i.e. Any object dtype columns following non object dtype (NaN valued) columns are comments
        comment_list=[]
        if(table[key].dtype!=object):#NaN valued column detector
            comment_flag=True
        if(comment_flag and table[key].dtype==object):#Checks for any object dtype columns thereafter
            comment_list.append(table[key])
        if(len(comment_list)>0):
            comment_list=list(pd.concat(comment_list).get_values())
            for i in range(0,l):
                microconcept_attributes[i]["comment"]=comment_list[i]
    initialise_level(microconcept_identifiers,"MicroConcepts",microconcept_names,concept_names,microconcept_attributes,
                     "Child",obj_dict,G)
    
    #Now define the Pre-Requisites between Concepts
    table=pd.read_csv("Funtoot/KG_%s/SubConceptDependency-Table 1.csv"%(dirname),header=1)
    add_level_connections(list(table["Subconcept"]),list(table["Dependency"]),obj_dict,G)

    #And the Pre-Requisites between MicroConcepts
    table=pd.read_csv("Funtoot/KG_%s/SSC Dependency-Table 1.csv"%(dirname),header=1)
    add_level_connections(list(table["SSC"]),list(table["Dependency"]),obj_dict,G)

    #Now let us define the Misconception Class Category
    table=pd.read_csv("Funtoot/KG_%s/LDs-Table 1.csv"%(dirname),header=1)
    category_names=table["Class Category"].drop_duplicates().dropna()
    category_parents=[table["Sub Sub Concept"][i] for i in category_names.keys()]
    category_ids=["MCN"+str(i) for i in range(0,len(category_names))]
    category_names=list(category_names)
    initialise_level(category_ids,"Misconception Class",category_names,category_parents,None,"Induces",obj_dict,G)

    #Finally let us add the Miconceptions in a given category
    misconception_names=list(table["Error Category"].dropna())
    misconception_parents=list(table["Class Category"].dropna())
    misconception_identifiers=list(table["LD Code"].dropna())
    misconception_attributes=[{"Description":i} for i in list(table["Teacher / Parent Description"].dropna())]
    initialise_level(misconception_identifiers,"Misconceptions",misconception_names,misconception_parents,
                     misconception_attributes,"Child",obj_dict,G)

In [11]:
def parse_edge(temp):
    string=""
    x=G.node[temp[0]]["link"].identifier
    y=G.node[temp[1]]["link"].identifier
    string+="\t\t<edge source=\"%s\" target=\"%s\">\n"%(x,y)
    string+="\t\t\t<data key=\"category\">%s</data>\n"%(G[temp[0]][temp[1]]['category'])
    string+="\t\t</edge>\n"
    return string

In [12]:
f=open("funtootNumeracy1.3.xml","w",encoding="utf-8")
header="""<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<graphml xmlns=\"http://graphml.graphdrawing.org/xmlns\"
\txmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\"
\txsi:schemaLocation=\"http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd\">
\t<key id=\"labels\" for=\"node\" attr.name=\"labels\" attr.type=\"string\"/>
\t<key id=\"uid\" for=\"node\" attr.name=\"uid\" attr.type=\"string\"/>
\t<key id=\"name\" for=\"node\" attr.name=\"name\" attr.type=\"string\"/>
\t<key id=\"objectType\" for=\"node\" attr.name=\"objectType\" attr.type=\"string\"/>
\t<key id=\"description\" for=\"node\" attr.name=\"description\" attr.type=\"string\"/>
\t<key id=\"tags\" for=\"node\" attr.name=\"tags\" attr.type=\"string\"/>
\t<key id=\"category\" for=\"node\" attr.name=\"category\" attr.type=\"str\"/>
\t<graph id=\"G\" edgedefault=\"directed\">\n"""
footer="\t</graph>\n</graphml>"
f.write(header)
for i in G.nodes():
    f.write(G.node[i]["link"].parse_graphML())
for i in G.edges():
    f.write(parse_edge(i))
f.write(footer)
f.close()